In [30]:
import open3d as o3d
import numpy as np

In [31]:
def align_mesh_principal_axes(mesh):
    # Compute the covariance matrix of the vertices
    vertices = np.asarray(mesh.vertices)
    cov = np.cov(vertices.T)
    
    # Perform eigen decomposition to get the principal axes
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    
    # Sort the eigenvectors by eigenvalues in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, sorted_indices]
    
    # Align the principal axes to the global axes
    # Primary (longest) to X, Secondary to Z, Tertiary to Y
    target_axes = np.array([[1, 0, 0],  # X-axis
                            [0, 0, 1],  # Z-axis
                            [0, 1, 0]]) # Y-axis
    
    rotation_matrix = np.dot(eigenvectors, target_axes)

    if np.linalg.det(rotation_matrix) < 0:
        rotation_matrix[:, 2] *= -1

    mesh.rotate(rotation_matrix, center=(0, 0, 0))
    
    return mesh

In [32]:
def center_mesh_bb(mesh):
    # Compute the center of the bounding box
    bbox = mesh.get_axis_aligned_bounding_box()
    bbox_center = bbox.get_center()
    
    # Translate the mesh to the origin
    mesh.translate(-bbox_center)
    
    return mesh

In [33]:
def compute_centre(mesh):
    vertices = np.asarray(mesh.vertices)
    centre = np.mean(vertices, axis=0)
    print(f"vertices mean: {centre}")
    return centre

In [34]:
def align_mesh_orientation(mesh): # input mesh must be centered with its bounding box (bb)

    centre = compute_centre(mesh)

    if centre[1] < 0:
        if centre[2] < 0: 
            # rotate 180 degrees around x
            mesh.rotate(o3d.geometry.get_rotation_matrix_from_xyz([np.pi, 0, 0]), center=(0, 0, 0))
        else: 
            # rotate 180 degrees around z
            mesh.rotate(o3d.geometry.get_rotation_matrix_from_xyz([0, 0, np.pi]), center=(0, 0, 0))
    
    else:
        if centre[2] < 0:
            # rotate 180 degrees around y
            mesh.rotate(o3d.geometry.get_rotation_matrix_from_xyz([0, np.pi, 0]), center=(0, 0, 0))
        else:
            pass
    
    centre = compute_centre(mesh)

    return mesh

In [37]:
# Load the mesh
file_name = "0601" # 4 digits: first 2 are patient index, last 2 are scan index - 01 for lower, 02 for upper
input_file_path = f"E:\\OneDrive\\OneDrive - University of Cambridge\\Documents\\Coding\\DPS_hku\\data_new_65536\\Origin\\00{file_name}_origin.ply"
output_file_path = f"E:\\OneDrive\\OneDrive - University of Cambridge\\Documents\\Coding\\DPS_hku\\gum_removal\\align\\00{file_name}_origin.ply"
mesh = o3d.io.read_triangle_mesh(input_file_path)

# Align the mesh based on principal axes
mesh = align_mesh_principal_axes(mesh)

# Center the mesh based on bounding box
mesh = center_mesh_bb(mesh)

# Ensure correct orientation
mesh = align_mesh_orientation(mesh)

# Save the aligned mesh
o3d.io.write_triangle_mesh(output_file_path, mesh)
print("Mesh alignment and orientation correction complete. Aligned mesh saved as", output_file_path)

vertices mean: [-4.20554645  1.5127214  -5.7640815 ]
vertices mean: [4.20554645 1.5127214  5.7640815 ]
Mesh alignment and orientation correction complete. Aligned mesh saved as E:\OneDrive\OneDrive - University of Cambridge\Documents\Coding\DPS_hku\gum_removal\align\000601_origin.ply
